In [147]:
import pandas as pd     # Se carga la Libreria de pandas
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity


In [173]:
lista_generos= ['Action','Indie','RPG','Simulation','Adventure','Free to Play','Strategy','Massively Multiplayer','Casual','Sports','Racing']

In [174]:
# Se carga el dataset games desde el archivo csv
df_games= pd.read_csv('DataSet_tranformados/games.csv',delimiter = ',',encoding = "utf-8")
df_games = df_games.loc[df_games['genres'].isin(lista_generos)]
df_games

,genres,game_name,game_id,release_year
0,Action,Lost Summoner Kitty,761140,2018
1,Casual,Lost Summoner Kitty,761140,2018
2,Indie,Lost Summoner Kitty,761140,2018
3,Simulation,Lost Summoner Kitty,761140,2018
4,Strategy,Lost Summoner Kitty,761140,2018
...,...,...,...,...
74826,Indie,Russian Roads,610660,2018
74827,Racing,Russian Roads,610660,2018
74828,Simulation,Russian Roads,610660,2018
74829,Casual,EXIT 2 - Directions,658870,2017


In [168]:
# Se carga el dataset games desde el archivo csv
df_reviews= pd.read_csv('DataSet_tranformados/reviews.csv',delimiter = ',',encoding = "utf-8")
df_reviews.head()

,user_id,year,game_id,recommend,sentiment_analysis
0,76561197970982479,2011,1250,True,2
1,76561197970982479,2011,22200,True,1
2,76561197970982479,2011,43110,True,2
3,js41637,2014,251610,True,2
4,js41637,2013,227300,True,2


In [213]:
# Se carga el dataset games desde el archivo csv
df_items= pd.read_parquet('DataSet_tranformados/items.parquet')
df_items.head()

,user_id,game_id,game_name,playtime_forever
0,76561197970982479,10,Counter-Strike,6
1,76561197970982479,20,Team Fortress Classic,0
2,76561197970982479,30,Day of Defeat,7
3,76561197970982479,40,Deathmatch Classic,0
4,76561197970982479,50,Half-Life: Opposing Force,0


###########################################################################################################

In [260]:
df_game_dummies = pd.get_dummies(df_games, columns=['genres'], prefix='', prefix_sep='')
df_games_genre_dummy = df_game_dummies.groupby('game_id', as_index=False).sum()
df_dummies = pd.get_dummies(df_reviews, columns=['sentiment_analysis'], prefix='', prefix_sep='')
df_review_sent_dummy = df_dummies.groupby('game_id', as_index=False).sum()
df_union_games_review= pd.merge(df_games_genre_dummy, df_review_sent_dummy, on='game_id', how= 'inner')

df_items_Xgameid= df_items.groupby('game_id', as_index=False)['playtime_forever'].sum()
df_union_games_revs_items= pd.merge(df_union_games_review, df_items_Xgameid, on='game_id', how= 'inner')
df_union_games_revs_items = df_union_games_revs_items[df_union_games_revs_items['playtime_forever'] > 10000]
df_union_games_revs_items = df_union_games_revs_items.reset_index(drop=True)


In [261]:
df_games_recomend= df_union_games_revs_items[['game_id','game_name']]
df_games_recomend.to_csv('DataSet_tranformados/funcion_modelo_ML.csv', index=False) 

In [262]:
df_union_games_revs_items= df_union_games_revs_items.drop(['game_id','game_name','release_year','user_id','year',], axis=1) # Eliminar columnas innecesarias

In [263]:
df_union_games_revs_items

,Action,Adventure,Casual,Free to Play,Indie,Massively Multiplayer,RPG,Racing,Simulation,Sports,Strategy,recommend,0,1,2,playtime_forever
0,1,0,0,0,0,0,0,0,0,0,0,56,2,29,26,17107858
1,1,0,0,0,0,0,0,0,0,0,0,11,2,8,7,960524
2,1,0,0,0,0,0,0,0,0,0,0,3,0,1,3,756375
3,1,0,0,0,0,0,0,0,0,0,0,1,0,0,1,154424
4,1,0,0,0,0,0,0,0,0,0,0,3,0,3,1,726545
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2153,0,0,0,0,0,0,0,1,1,0,0,1,0,0,1,15034
2154,0,0,0,0,1,0,0,0,1,0,1,2,0,0,2,14891
2155,0,1,0,0,0,0,0,0,0,0,0,8,0,2,7,26698
2156,0,0,1,0,0,0,0,0,0,0,0,1,0,1,0,20259


In [264]:
escalar= StandardScaler()
df_escalado = escalar.fit_transform(df_union_games_revs_items)
similitudes= cosine_similarity(df_escalado)

In [265]:
similitudes.shape

(2158, 2158)

In [266]:
def recomendacion_juego(id_game: int):
    result= {}
    lista_juegos= []    
    indice = df_games_recomend.loc[df_games_recomend['game_id'] == id_game].index
    if indice.empty:
        result= {"Error": 'JUEGO NO ENCONTRADO'}
    else:
        indice= indice[0]
        distancias= sorted(list(enumerate(similitudes[indice])), reverse= True, key=lambda x:x[1])
        for item in distancias[1:6]:
            lista_juegos.append(df_games_recomend.iloc[item[0]]['game_name'])
        result = {'JUEGOS RECOMENTADOS': lista_juegos}
    return result

In [267]:
recomendacion_juego(50)

{'JUEGOS RECOMENTADOS': ['STAR WARS™ Jedi Knight II - Jedi Outcast™',
  'Defence Alliance 2',
  'Unreal Tournament 3 Black',
  'Crysis Warhead®',
  'Red Orchestra: Ostfront 41-45']}

In [268]:
np.save('DataSet_tranformados/modelo.npy', similitudes)